In [159]:
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import numpy as np
import scipy
from numpy import linalg as LA

In [251]:
# 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.

data_train = pd.read_csv('salary-train.csv')
data_test =  pd.read_csv('salary-test-mini.csv')

In [252]:
# 2. Проведите предобработку:
#   • Приведите тексты к нижнему регистру.

data_train.loc[:, 'FullDescription'] = data_train['FullDescription'].str.lower()
data_test.loc[:, 'FullDescription'] = data_test['FullDescription'].str.lower()

#   • Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. 
#   Для такой замены в строке text подходит следующий вызов:
#   re.sub(’[^a−zA−Z0−9]’, ’␣’, text.lower())

data_train.loc[:, 'FullDescription'] = data_train['FullDescription'].str.replace('\W', ' ')
data_test.loc[:, 'FullDescription'] = data_test['FullDescription'].str.replace('\W', ' ')

<ipython-input-252-b6886f0c739b>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data_train.loc[:, 'FullDescription'] = data_train['FullDescription'].str.replace('\W', ' ')
<ipython-input-252-b6886f0c739b>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data_test.loc[:, 'FullDescription'] = data_test['FullDescription'].str.replace('\W', ' ')


In [257]:
#   • Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
#   Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 5)
X_text = vectorizer.fit_transform(data_train.FullDescription)
X_test_text = vectorizer.transform(data_test.FullDescription)

In [258]:
#   • Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку ’nan’. 
#   Код для этого был приведен выше.
from sklearn.feature_extraction import DictVectorizer

#   • Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

v = DictVectorizer()

data_train['LocationNormalized'].fillna('nan', inplace=True) 
data_train['ContractTime'].fillna('nan', inplace=True)

X_train_categ = v.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict ('records'))
X_test_categ = v.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict ('records'))


#   • Объедините все полученные признаки в одну матрицу "объекты - признаки". 
#   Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. 
#   Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

from scipy.sparse import hstack
X = hstack([X_text, X_train_categ])
X_test = hstack([X_test_text, X_test_categ])


In [259]:
# 3. Обучите гребневую регрессию с параметром alpha=1. 
# Целевая переменная записана в столбце SalaryNormalized.
from sklearn.linear_model import Ridge

y = data_train['SalaryNormalized']
clf = Ridge(alpha = 1)
clf.fit(X, y)
# 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. 
# Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

clf.predict(X_test)

array([56581.48527652, 37156.00502328])

In [260]:
data_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN
